In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Index, Document, UpdateByQuery

from ssl import create_default_context
import json

In [ ]:
# ! docker cp searchapi-elasticsearch-1:usr/share/elasticsearch/config/certs/ca/ca.crt .
# ! docker exec -it fastapi-elasticsearch-1 /usr/share/elasticsearch/bin/elasticsearch-reset-password -u elastic
# ! docker exec -it fastapi-elasticsearch-1 /usr/share/elasticsearch/bin/elasticsearch-create-enrollment-token -s kibana
# ! docker exec -it fastapi-kibana-1 /usr/share/kibana/bin/kibana-verification-code

In [ ]:
! docker cp searchapi_elasticsearch_1:usr/share/elasticsearch/config/certs/ca/ca.crt .

In [5]:
#context = create_default_context(cafile='./ca.crt')

es = Elasticsearch('https://es01:9200', basic_auth=('elastic','elastic'),
    #ssl_context=context,
)

In [6]:
es

<Elasticsearch([{'host': 'es01', 'port': 9200, 'use_ssl': True}])>

In [5]:
with open('database.json') as json_file:
    database = json.load(json_file)

# Transfer string to dict    
database = json.loads(database)

for key, values in database.items():
    print(key)
    for item in values:
        print(item)

user
{'name': 'Tibbers Hao', 'affiliation': 'LBNL', 'role': 'admin', 'uid': 'A12345'}
{'name': 'Tanny Andrea Chavez Esparza', 'affiliation': 'LBNL', 'role': 'admin', 'uid': 'A67890'}
{'name': 'Zhuowen Zhao', 'affiliation': 'LBNL', 'role': 'admin', 'uid': 'A13579'}
{'name': 'Elizabeth Holman', 'affiliation': 'LBNL', 'role': 'admin', 'uid': 'A24680'}
{'name': 'Hari Krishnan', 'affiliation': 'LBNL', 'role': 'admin', 'uid': 'A09876'}
{'name': 'Howard Yanxon', 'affiliation': 'ANL', 'role': 'admin', 'uid': 'A54321'}
{'name': 'Dula Parkinson', 'affiliation': 'LBNL', 'role': 'user', 'uid': 'B12121'}
{'name': 'John Doe', 'affiliation': 'UCB', 'role': 'user', 'uid': 'B56565'}
{'name': 'Jane Doe', 'affiliation': 'Stanford', 'role': 'user', 'uid': 'B78787'}
app
{'name': 'segmentation', 'target': 'image', 'uid': '1'}
{'name': 'peak detection', 'target': 'spectra', 'uid': '2'}
{'name': 'label maker', 'target': 'image', 'uid': '3'}
{'name': 'colorwheel', 'target': 'image', 'uid': '4'}


In [17]:
# Connection with Content Registry
import urllib.request

def model_list_GET_call(url):
    """
    Get the whole model registry data from the fastapi url.
    """
    #url = 'http://localhost:8000/api/v0/models'  # current host, could be inside the docker
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data

In [10]:
keys = ["name","version","type","uri","reference", "description","content_id"]

for model in model_list_GET_call():
    content_data = {}
    for key, value in model.items():
        if key in keys:
            content_data[key] = value
    # does not support list as dict value and [_id], as it's been used for elasticsearch
    es.index(index = 'model', id = content_data['content_id'], document = content_data)

NameError: name 'model_list_GET_call' is not defined

In [26]:
url_head = 'http://localhost:8000/api/v0/'
catagory = ['models', 'apps', 'workflows']

In [27]:
for item in catagory:
    url = url_head + item
    print(url)

http://localhost:8000/api/v0/models
http://localhost:8000/api/v0/apps
http://localhost:8000/api/v0/workflows


In [29]:
keys = ["name", "version", "type", "uri", "application", "reference", "description", "content_type", "content_id", "owner"]

url_head = 'http://localhost:8000/api/v0/'
catagory = ['models', 'apps', 'workflows']

for item in catagory:
    url = url_head + item

    for model in model_list_GET_call(url):
        
        content_data = {}
        for key, value in model.items():
            if key in keys:
                content_data[key] = value
        print(content_data['content_type'])
        print(content_data['content_id'])        
        print(content_data)



model
9be3d982-b6cd-4403-8373-c0e66f2eb98a
{'content_type': 'model', 'name': 'pytorch_autoencoder', 'version': '0.0.1', 'type': 'unsupervised', 'owner': 'mlexchange team', 'uri': 'mlexchange/unsupervised-classifier', 'application': ['data_clinic'], 'description': 'xxx', 'content_id': '9be3d982-b6cd-4403-8373-c0e66f2eb98a'}
model
a5e21579-c238-440a-8703-3f858e2ddbc3
{'version': '1.0.0', 'type': 'unsupervised', 'uri': 'xxx', 'application': ['segmentation', 'clustering'], 'description': 'xxx', 'reference': 'Nicholas Schwartz & Howard Yanxon', 'content_type': 'model', 'content_id': 'a5e21579-c238-440a-8703-3f858e2ddbc3', 'name': 'kmeans', 'owner': 'mlexchange team'}
model
dd184a4d-beee-4f1c-9ce1-3b03a6eac160
{'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'application': ['classification', 'segmentation'], 'description': 'xxx', 'reference': 'Peter Zwart & Eric Roberts', 'content_type': 'model', 'content_id': 'dd184a4d-beee-4f1c-9ce1-3b03a6eac160', 'name': 'pyMSDtorch', 'owner': 'ml

In [10]:
# Insert docs
for key, values in database.items():
    for item in values:
        flag = True
        try: 
            record = es.get(index = key, id = item['uid'])
        except Exception as e:
            flag = False
            print('uid: ' + item['uid'] + ' not exists, inserting ...') 
        
        if not flag:
            es.index(index = key, id = item['uid'], document = item)

# FastAPI Callback List


## Search doc based on index

HTTP GET

URI: ‘/search-doc/{index}/{field}’

Def search_doc(index: Optional[str], field: str)

Description: Search the doc information based on input index and field

Return: `{index} : {field}`


In [7]:
def search(keyword: str):
    '''
    Search the doc information based on input name.
    To do: add sort for ordered display
    '''
    resp = Search().using(es).query("multi_match", query = keyword, fuzziness = "AUTO").extra(track_total_hits = True).execute()
    result = resp.to_dict()
    #print('Total %d hits found: ' % resp.hits.total.value)
    return ['Total %d hits found: ' % resp.hits.total.value, resp]
    #for hit in resp:
    #    print('Category: ' + hit.meta.index, hit)


In [8]:
search('unsupervised')

ConnectionError: ConnectionError(<urllib3.connection.HTTPSConnection object at 0x7fca608eb820>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known) caused by: NewConnectionError(<urllib3.connection.HTTPSConnection object at 0x7fca608eb820>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known)

In [33]:
from elasticsearch_dsl.query import MultiMatch, Match

# {"multi_match": {"query": "python django", "fields": ["title", "body"]}}
MultiMatch(query='user')


MultiMatch(query='user')

In [34]:
from elasticsearch_dsl import Q

Q("multi_match", query='user')

MultiMatch(query='user')

In [42]:
search('model')

['Total 7 hits found: ',
 <Response: [<Hit(model/9be3d982-b6cd-4403-8373-c0e66f2eb98a): {'content_type': 'model', 'name': 'pytorch_autoencoder', 've...}>, <Hit(model/a5e21579-c238-440a-8703-3f858e2ddbc3): {'version': '1.0.0', 'type': 'unsupervised', 'uri': 'xxx', '...}>, <Hit(model/dd184a4d-beee-4f1c-9ce1-3b03a6eac160): {'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'ap...}>, <Hit(model/d68d176e-3c9e-4cb5-8d86-e775164c9dbd): {'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'ap...}>, <Hit(model/37d9c5c0-a044-427d-b722-6ddef878f59b): {'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'ap...}>, <Hit(model/1705cb31-f030-401f-8fbc-791eeec2c445): {'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'ap...}>, <Hit(model/d39d5efc-aa47-4821-a145-b594df677e9d): {'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'ap...}>]>]

In [58]:
result

{'query': {'bool': {'filter': [{'term': {'category': 'name'}}],
   'must': [{'match': {'title': 'TF-NeutralNetworks'}}]}}}

In [60]:
resp = Search().using(es).query('match', name = 'pytorch_autoencoder').extra(track_total_hits = True).execute()

print(resp.success())

print(resp.took)

print(resp.hits.total.relation)

print(resp.hits.total.value)


# for h in resp:
#     print(h.name, h.uid)


resp.to_dict()


True
4
eq
1


{'took': 4,
 'timed_out': False,
 '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 2.0167708,
  'hits': [{'_index': 'model',
    '_id': '9be3d982-b6cd-4403-8373-c0e66f2eb98a',
    '_score': 2.0167708,
    '_source': {'name': 'pytorch_autoencoder',
     'version': '0.0.1',
     'type': 'unsupervised',
     'uri': 'mlexchange/unsupervised-classifier',
     'description': 'xxx',
     'content_id': '9be3d982-b6cd-4403-8373-c0e66f2eb98a'}}]}}

## Add a new index

HTTP POST

URI: ‘/add-index/{index}’

Def add_index(index: str)

Description: Check if the index exists, if not, add the given index

Return: `{index} has been successfully inserted!`


In [15]:
def create_index(index: str):
    '''
    Check if the index exists, if not, add the given index
    '''
    try:
        resp = Index(index).create(using = es)
        return [f'Index: \"{index}\" has been successfully created!', resp]

    except Exception as e:
        return f'Index: \"{index}\" already exists, please check.'

    

In [16]:
create_index('model')

'Index: "model" already exists, please check.'

## Add/update doc into an index

HTTP POST

URI: ‘/add-doc/{index}/{field}’

Def add_doc(index: str, field: str)

Description: Check if the doc exists, if not, add doc to the given index

Return: `{field} has been successfully inserted into {index}!`


In [30]:
def create_doc(index: str, id: str, doc: dict):
    '''
    Check if the doc exists within the given index, then add/update doc to the index.
    '''
    check_index = Index(index).exists(using = es)
    if not check_index:
        return f'Index: \"{index}\" does not exist, please check.'
    else:
        check_doc = Document().exists(using = es, index = index, id = id)
        if check_doc:
            es.index(index = index, id = id, document = doc)
            return 'UID: \"' + doc['uid'] + '\" has been updated.'
        else:
            es.index(index = index, id = id, document = doc)
            return f'New document has been successfully inserted in \"{index}\".'



In [31]:
doc = {'name': 'Tibbers Hao',
   'affiliation': 'LBNL',
   'role': 'admin',
   'uid': 'A12345'}
create_doc('user', doc['uid'], doc)

'UID: "A12345" has been updated.'

## Delete a doc

HTTP DELETE

URI: ‘/delete-doc/{index}/{id or field}’

Def delete_doc(index: str, field: str, id: str/int)

Description: Check if the doc exists, if yes, delete doc in the specified index

Return: ‘{field} has been successfully Deleted in {index}!’ 


In [477]:
def delete_doc(index: str, keyword: str):
    '''
    Delete document within specified index
    '''
    resp = Search().using(es).index(index).query('match', name = keyword).delete()
    return 'Document has been deleted.'

In [485]:
delete_doc('user', '')

'Document has been deleted.'

## Delete an index 

HTTP DELETE

URI: ‘/delete-index/{index}’

Def delete_index(index; str)

Description: Check if the index exists, if yes, delete the index (and all docs within)

Return: ‘{index} has been successfully Deleted!’ 


In [66]:
def delete_index(index: str):
    '''
    Check if the index exists, if not, add the given index
    '''
    try:
        resp = Index(index).delete(using = es)
        print(f'Index: \"{index}\" has been successfully deleted!')
        print(resp)
    except Exception as e:
        print(f'Index: \"{index}\" does not exist.')

In [74]:
delete_index('model')

Index: "model" has been successfully deleted!
{'acknowledged': True}


In [32]:
for h in Search().using(es).scan():
    print(h)

<Hit(app/1): {'name': 'segmentation', 'target': 'image', 'uid': '1'}>
<Hit(model/9be3d982-b6cd-4403-8373-c0e66f2eb98a): {'content_type': 'model', 'name': 'pytorch_autoencoder', 've...}>
<Hit(user/A12345): {'name': 'Tibbers Hao', 'affiliation': 'LBNL', 'role': 'admi...}>
<Hit(workflow/ad5e2cd0-ac88-4c31-bc05-65dbd4ca3cd9): {'content_type': 'workflow', 'name': 'Image Labeling', 'vers...}>
<Hit(app/2): {'name': 'peak detection', 'target': 'spectra', 'uid': '2'}>
<Hit(model/a5e21579-c238-440a-8703-3f858e2ddbc3): {'version': '1.0.0', 'type': 'unsupervised', 'uri': 'xxx', '...}>
<Hit(user/A67890): {'name': 'Tanny Andrea Chavez Esparza', 'affiliation': 'LBNL...}>
<Hit(app/3): {'name': 'label maker', 'target': 'image', 'uid': '3'}>
<Hit(model/dd184a4d-beee-4f1c-9ce1-3b03a6eac160): {'version': '1.0.0', 'type': 'supervised', 'uri': 'xxx', 'ap...}>
<Hit(user/A13579): {'name': 'Zhuowen Zhao', 'affiliation': 'LBNL', 'role': 'adm...}>
<Hit(app/4): {'name': 'colorwheel', 'target': 'image', 'uid': '4'

In [44]:
str(['a', 'b'])

"['a', 'b']"